# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

# Parse First Text File

In [2]:
file_path = 'mathwell_questions2.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100) #past percentage was 54.88
print(result1)
print(len(df1), total_questions)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
Percentage of Questions with Executable Code: 55.395472995209715
15149 27347


In [3]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 6.9311505709947845


In [4]:
df1 = df1.drop_duplicates(subset='question')

In [5]:
df1

,question,solution,answer,topic
0,A baker baked 120 cakes. He sold 40 on the fir...,def solution():\n\n # The baker started wit...,30,Animals and Wildlife
1,Naruto has 300 ramen noodles. He eats 20 ramen...,def solution():\n #Naruto has 300 ramen noo...,200,Naruto
2,A 5th-grade class is doing a science project. ...,def solution():\n #20 students are in the c...,60,DIY Science Projects
3,A mummy is 3000 years old. \n\nHis sarcophagus...,def solution():\n #The mummy is 3000 years ...,4000,Mummies
4,The volleyball team has 12 players. \n4 of the...,def solution():\n #Total number of players\...,0,volleyball
...,...,...,...,...
15143,A phoenix is a mythical bird that is reborn fr...,def solution():\n #The first phoenix was bo...,1005500,Mythology
15144,"Ratatouille, the rat, ate 12 slices of pizza. ...",def solution():\n #Ratatouille ate 12 slice...,0,Ratatouille
15145,A detective is investigating a murder. He inte...,def solution():\n #Total people interviewed...,20,Detectives and Mystery
15146,Shrek ate 30 onions. 10 of the onions were gre...,def solution():\n #Shrek ate 30 onions\n ...,0,Shrek


# Parse Second Text File

In [6]:
file_path = 'mathwell_questions_no_topic.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
print(result1)
print(len(df2), total_questions)

Percentage of Questions with Executable Code: 46.500577886220626
7242 15574


In [7]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 4.901960784313726


In [8]:
df2 = df2.drop_duplicates(subset='question')

In [9]:
df2

,question,solution,answer,topic
0,2000 people in a stadium. 1/4 of the people ar...,def solution():\n #Total people in the stad...,850.0,unspecified
1,The first 500 people in line for the Comic-Con...,def solution():\n #The first 500 people in ...,1000,unspecified
2,There are 120000 spiders in a spider farm. 100...,def solution():\n #There were 120000 spider...,100000,unspecified
3,A baker bakes 100 pies. He sells 20 of them on...,def solution():\n #Baker bakes 100 pies\n ...,10,unspecified
4,A 747 can hold 400 people. \nA 777 can hold 30...,def solution():\n #A 747 can hold 400 peopl...,4000,unspecified
...,...,...,...,...
7232,400 people are at a rally. 200 of them are wea...,def solution():\n #Total people at the rall...,100,unspecified
7233,20000 people live in a small town. 2000 of the...,def solution():\n #12000 people live in a s...,0,unspecified
7237,A baker is baking 1000 cookies. He bakes 200 c...,def solution():\n #The baker is baking 1000...,-400,unspecified
7238,200 people are at a rally. 1/4 of them are wea...,def solution():\n\n #Total number of people...,<class 'int'>,unspecified


# Concat DFs

In [10]:
extra_qs = pd.read_csv('data/extra_qs.csv')
df = pd.concat([df1, df2, extra_qs])
df = df.drop_duplicates(subset='question')
df = df.sample(frac = 1, random_state = 42)
df = df[['question', 'solution', 'answer', 'topic']]
#df = df[:17692]
#df.to_csv("data/sgsm_unannotated.csv")

In [11]:
df ## last run had 17692

,question,solution,answer,topic
10739,The Egyptians built 100 pyramids. 20 of them a...,def solution():\n #Total number of pyramids...,15,Ancient Civilizations
5852,A 2-liter bottle of soda has 80 calories. A 1....,def solution():\n #A 2-liter bottle of soda...,400,Health and Nutrition
1910,There are 20000 apples in an orchard. 2000 of ...,def solution():\n #There are 20000 apples i...,15000,unspecified
7809,There are 1000 pages in a Mystery Stories book...,def solution():\n #Total pages in the book\...,500,Mystery Stories
14961,1000 Minions are in a room. 200 of them are we...,def solution():\n #1000 Minions are in a ro...,1000,Minions
...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)"


In [12]:
#df = pd.read_csv('data/sgsm_unannotated.csv')
def parse_solution(df):
    if "\nBe" in df['solution']:
        solution = df['solution'].split("\nBe")[0]
    if "##" in df['solution']:
        solution = df['solution'].split("##")[0]
    else:
        solution = df['solution']
    return solution
df['solution'] = df.apply(parse_solution, axis =1)
df.to_csv("data/sgsm_unannotated.csv")
sgsm_train = pd.read_csv('data/sgsm_train.csv')
# sgsm_train = sgsm_train.drop_duplicates(subset= 'question')
# sgsm_train.to_csv('data/sgsm_train.csv')
sgsm_train['subset'] = 'sgsm_train'
df['subset'] = 'sgsm_unannotated'
sgsm = pd.concat([sgsm_train, df])
sgsm = sgsm[['question', 'solution', 'answer', 'topic', 'subset']]
#sgsm.to_csv('data/sgsm.csv')
#sgsm = sgsm.drop_duplicates(subset = ['question', 'subset'])
sgsm

,question,solution,answer,topic,subset
0,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream,sgsm_train
1,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball,sgsm_train
2,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman,sgsm_train
3,3 scoops of ice cream cost $4. How much does ...,def solution():\n #3 scoops of ice cream co...,133.33333333333334,ice cream,sgsm_train
4,Superman can fly 1000 miles per hour. He flies...,def solution():\n #Superman can fly 1000 mi...,1.5,Superman,sgsm_train
...,...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts,sgsm_unannotated
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures,sgsm_unannotated
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens,sgsm_unannotated
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)",sgsm_unannotated


In [13]:
sgsm = sgsm.drop_duplicates(subset = 'question')
sgsm

,question,solution,answer,topic,subset
0,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream,sgsm_train
1,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball,sgsm_train
2,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman,sgsm_train
3,3 scoops of ice cream cost $4. How much does ...,def solution():\n #3 scoops of ice cream co...,133.33333333333334,ice cream,sgsm_train
4,Superman can fly 1000 miles per hour. He flies...,def solution():\n #Superman can fly 1000 mi...,1.5,Superman,sgsm_train
...,...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts,sgsm_unannotated
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures,sgsm_unannotated
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens,sgsm_unannotated
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)",sgsm_unannotated


In [14]:
sgsm.to_csv('data/sgsm.csv')
sgsm.to_csv('sgsm.csv')
sgsm['subset'].value_counts()

subset
sgsm_unannotated    18397
sgsm_train           2092
Name: count, dtype: int64

In [15]:
sgsm_train = pd.read_csv('data/sgsm_train.csv')
sgsm_train.drop_duplicates(subset= ['question', 'solution'])

,Unnamed: 0,question,solution,answer,topic
0,1,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream
1,3,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball
2,4,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman
3,5,3 scoops of ice cream cost $4. How much does ...,def solution():\n #3 scoops of ice cream co...,133.33333333333334,ice cream
4,6,Superman can fly 1000 miles per hour. He flies...,def solution():\n #Superman can fly 1000 mi...,1.5,Superman
...,...,...,...,...,...
2087,1901,30 unicorns are in a herd. 10 unicorns in the ...,def solution():\n #Total number of unicorns...,17,unicorns
2088,1902,A pizza has 8 slices. 4 of the slices have pep...,def solution():\n #A pizza has 8 slices\n ...,4,pizza
2089,1903,12000 people were at a rally. 10000 of them we...,def solution():\n #Total number of people a...,500,unspecified
2090,1904,A bakery made 300 muffins. 150 of the muffins ...,def solution():\n #Total muffins\n total...,60,unspecified


In [16]:
sgsm_train

,Unnamed: 0,question,solution,answer,topic
0,1,3 scoops of ice cream cost $4. \nHow much does...,def solution():\n #3 scoops of ice cream co...,13.333333333333332,ice cream
1,3,A volleyball team is 6 players. \n3 of the pla...,def solution():\n #3 of the players are 16 ...,16.666666666666668,volleyball
2,4,"Superman can fly at a speed of 1,000 miles per...",def solution():\n #Superman can fly at a sp...,0.2,Superman
3,5,3 scoops of ice cream cost $4. How much does ...,def solution():\n #3 scoops of ice cream co...,133.33333333333334,ice cream
4,6,Superman can fly 1000 miles per hour. He flies...,def solution():\n #Superman can fly 1000 mi...,1.5,Superman
...,...,...,...,...,...
2087,1901,30 unicorns are in a herd. 10 unicorns in the ...,def solution():\n #Total number of unicorns...,17,unicorns
2088,1902,A pizza has 8 slices. 4 of the slices have pep...,def solution():\n #A pizza has 8 slices\n ...,4,pizza
2089,1903,12000 people were at a rally. 10000 of them we...,def solution():\n #Total number of people a...,500,unspecified
2090,1904,A bakery made 300 muffins. 150 of the muffins ...,def solution():\n #Total muffins\n total...,60,unspecified


In [17]:
df

,question,solution,answer,topic,subset
10739,The Egyptians built 100 pyramids. 20 of them a...,def solution():\n #Total number of pyramids...,15,Ancient Civilizations,sgsm_unannotated
5852,A 2-liter bottle of soda has 80 calories. A 1....,def solution():\n #A 2-liter bottle of soda...,400,Health and Nutrition,sgsm_unannotated
1910,There are 20000 apples in an orchard. 2000 of ...,def solution():\n #There are 20000 apples i...,15000,unspecified,sgsm_unannotated
7809,There are 1000 pages in a Mystery Stories book...,def solution():\n #Total pages in the book\...,500,Mystery Stories,sgsm_unannotated
14961,1000 Minions are in a room. 200 of them are we...,def solution():\n #1000 Minions are in a ro...,1000,Minions,sgsm_unannotated
...,...,...,...,...,...
12196,The Ringling Bros. and Barnum & Bailey Circus ...,def solution():\n #Total number of clowns\n...,50,the Circus and Performing Arts,sgsm_unannotated
12932,A unicorn has 2000 magical hairs. 1000 of thes...,def solution():\n #A unicorn has 2000 magic...,500,Mythical Creatures,sgsm_unannotated
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens,sgsm_unannotated
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)",sgsm_unannotated


In [18]:
df['topic'].value_counts()

topic
unspecified                                          4383
National Parks                                        159
LeBron James                                          157
Dance and Ballet                                      154
cheer leading                                         154
                                                     ... 
Transformers                                           18
Up                                                     18
Pokemon                                                13
Hobbies (e.g., stamp collecting, coin collecting)      10
Pets (cats, dogs, hamsters)                             6
Name: count, Length: 198, dtype: int64

In [19]:
17692+2094

19786

In [20]:
import random
for i in range(0, 10):
    index = random.randint(0, len(df))
    question = df.iloc[index]['question']
    solution = df.iloc[index]['solution']
    print(f"Question: {question}\n\nSolution:\n{solution}")

Question: A robotics company has 1000 robots. 200 of them are in the warehouse, 300 of them are in the lab, and the rest are in the showroom. How many robots are in the showroom?

Solution:
def solution():
    #Total number of robots
    total_robots = 1000
    #Number of robots in the warehouse
    robots_warehouse = 200
    #Number of robots in the lab
    robots_lab = 300
    #Number of robots in the showroom
    robots_showroom = total_robots - robots_warehouse - robots_lab
    return robots_showroom
Question: Thomas the Tank Engine is a train. He can pull 1000 cars. 200 of the cars are in the first train. 300 of the cars are in the second train. 500 of the cars are in the third train. How many cars are left to be pulled?

Solution:
def solution():
    #Thomas the Tank Engine can pull 1000 cars
    thomas_cars = 1000
    #200 of the cars are in the first train
    first_train_cars = 200
    #300 of the cars are in the second train
    second_train_cars = 300
    #500 of the cars ar